## Project 2 Final Project Submission

Please fill out:
* Student name: Elijah Jarocki
* Student pace: full time
* Scheduled project review date/time: March 1
* Instructor name: Jelly Spratley
* Blog post URL: _______


# Overview and Business Understanding
My stakeholder is Lynn Crane Real Estate. She wanted to identify trends within home sales in King County, Washington. There are many considerations when buying and selling a home that she must take into account. She wanted to know which attributes of a home are the most desirable (ie. yield the highest selling price). I used data from the King County House Sales dataset to create several models to determine which features affected price the most. 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tools.eval_measures import rmse
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy import stats

# Data Preparation
Before beginning to model the data, I did some exploratory data analysis to glean some initial direction for the project. Below are listed the meanings of each feature within the dataset. Building condition and grade have more detailed explataions below. 

# Column Names and Descriptions for King County Data Set
`id` - Unique identifier for a house
* `date` - Date house was sold
* `price` - Sale price (prediction target)
* `bedrooms` - Number of bedrooms
* `bathrooms` - Number of bathrooms
* `sqft_living` - Square footage of living space in the home
* `sqft_lot` - Square footage of the lot
* `floors` - Number of floors (levels) in house
* `waterfront` - Whether the house is on a waterfront
  * Includes Duwamish, Elliott Bay, Puget Sound, Lake Union, Ship Canal, Lake Washington, Lake Sammamish, other lake, and river/slough waterfronts
* `view` - Quality of view from house
  * Includes views of Mt. Rainier, Olympics, Cascades, Territorial, Seattle Skyline, Puget Sound, Lake Washington, Lake Sammamish, small lake / river / creek, and other
* `condition` - How good the overall condition of the house is. Related to maintenance of house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each condition code
* `grade` - Overall grade of the house. Related to the construction and design of the house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each building grade code
* `sqft_above` - Square footage of house apart from basement
* `sqft_basement` - Square footage of the basement
* `yr_built` - Year when house was built
* `yr_renovated` - Year when house was renovated
* `zipcode` - ZIP Code used by the United States Postal Service
* `lat` - Latitude coordinate
* `long` - Longitude coordinate
* `sqft_living15` - The square footage of interior housing living space for the nearest 15 neighbors
* `sqft_lot15` - The square footage of the land lots of the nearest 15 neighbors

	BUILDING CONDITION Relative to age and grade. Coded 1-5.

1 = Poor- Worn out. Repair and overhaul needed on painted surfaces, roofing, plumbing, heating and numerous functional inadequacies. Excessive deferred maintenance and abuse, limited value-in-use, approaching abandonment or major reconstruction; reuse or change in occupancy is imminent. Effective age is near the end of the scale regardless of the actual chronological age.

2 = Fair- Badly worn. Much repair needed. Many items need refinishing or overhauling, deferred maintenance obvious, inadequate building utility and systems all shortening the life expectancy and increasing the effective age.

3 = Average- Some evidence of deferred maintenance and normal obsolescence with age in that a few minor repairs are needed, along with some refinishing. All major components still functional and contributing toward an extended life expectancy. Effective age and utility is standard for like properties of its class and usage.

4 = Good- No obvious maintenance required but neither is everything new. Appearance and utility are above the standard and the overall effective age will be lower than the typical property.

5= Very Good- All items well maintained, many having been overhauled and repaired as they have shown signs of wear, increasing the life expectancy and lowering the effective age with little deterioration or obsolescence evident with a high degree of utility.

BUILDING GRADE

    	Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 Falls short of minimum building standards. Normally cabin or inferior structure.

4 Generally older, low quality construction. Does not meet code.

5 Low construction costs and workmanship. Small, simple design.

6 Lowest grade currently meeting building code. Low quality materials and simple designs.

7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

In [4]:
data = pd.read_csv("data/kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [6]:
corr = data.corr()
corr["price"].sort_values()

zipcode         -0.053402
id              -0.016772
long             0.022036
yr_built         0.053953
sqft_lot15       0.082845
sqft_lot         0.089876
yr_renovated     0.129599
floors           0.256804
lat              0.306692
bedrooms         0.308787
bathrooms        0.525906
sqft_living15    0.585241
sqft_above       0.605368
sqft_living      0.701917
price            1.000000
Name: price, dtype: float64

As demonstrated above, the price is most highly correlated with sqft_living, sqft_above, sqft_living15, and bathrooms. 

We can also drop the following features as they have a low correlation with price: long, sqft_basement, yr_rennovated, zipcode, lat, 

My first dataframe, DF1 consists of all the numeric features within the dataset. We will worry about categorical variables later on. 

In [11]:
df1 = data.drop(["date", "id", "waterfront", "view", "condition", "grade",
                "sqft_basement", "yr_renovated"], axis=1) 
df1.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   floors         21597 non-null  float64
 6   sqft_above     21597 non-null  int64  
 7   yr_built       21597 non-null  int64  
 8   zipcode        21597 non-null  int64  
 9   lat            21597 non-null  float64
 10  long           21597 non-null  float64
 11  sqft_living15  21597 non-null  int64  
 12  sqft_lot15     21597 non-null  int64  
dtypes: float64(5), int64(8)
memory usage: 2.1 MB


# Modeling

Linear Regression is a way of modeling an independent variable against a dependent variable. By creating a Linear Regression model, we can take features from a potential home purchase and estimate a sale price for the home. If the house is being sold under value, this is a good investment for Lynn Crane Real Estate.

In [12]:
y = df1["price"]
x = df1[["bedrooms", "bathrooms", "sqft_living", "sqft_lot", 
        "floors", "sqft_above", "yr_built", "zipcode", "lat", "long",
        "sqft_living15", "sqft_lot15"]]

In [52]:
model = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
model_summary = model.summary()
model_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     3022.
Date:                Mon, 28 Feb 2022   Prob (F-statistic):               0.00
Time:                        21:44:38   Log-Likelihood:            -2.9675e+05
No. Observations:               21597   AIC:                         5.935e+05
Df Residuals:                   21584   BIC:                         5.936e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.133e+07    3.2e+06     -3.543      0.000   -1.76e+07   -5.06e+06
bedrooms      -5.873e+04   2083.354    -28.189      0.000   -6.28e+04   -5.46e+04
bathrooms      6.241e+04   3594.234     17.365      0.000    5.54e+04    6.95e+04
sqft_living     225.7676      4.724     47.795      0.000     216.509     235.026
sqft_lot          0.1940      0.053      3.633      0.000       0.089       0.299
floors         2.614e+04   3983.921      6.561      0.000    1.83e+04    3.39e+04
sqft_above       30.0593      4.740      6.342      0.000      20.769      39.349
yr_built      -2591.7602     71.381    -36.309      0.000   -2731.671   -2451.849
zipcode        -573.3922     36.399    -15.753      0.000    -644.737    -502.047
lat            6.247e+05   1.17e+04     53.193      0.000    6.02e+05    6.48e+05
long          -3.505e+05   1.45e+04    -24.252      0.000   -3.79e+05   -3.22e+05
sqft_living15    86.8970      3.614     24.043      0.000      79.813      93.981
sqft_lot15       -0.3965      0.082     -4.849      0.000      -0.557      -0.236
==============================================================================
Omnibus:                    17457.545   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1150905.525
Skew:                           3.424   Prob(JB):                         0.00
Kurtosis:                      38.101   Cond. No.                     2.11e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
vif_data = pd.DataFrame()
vif_data["feature"] = x.columns
vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]
print(vif_data)

          feature           VIF
0        bedrooms  2.258289e+01
1       bathrooms  2.806096e+01
2     sqft_living  4.861298e+01
3        sqft_lot  2.371439e+00
4          floors  1.650173e+01
5      sqft_above  3.665083e+01
6        yr_built  7.560934e+03
7         zipcode  1.566040e+06
8             lat  1.335540e+05
9            long  1.326766e+06
10  sqft_living15  2.423291e+01
11     sqft_lot15  2.592180e+00


I surmised from the initial OLS model that there does exist SOME relationship between these variables and price, but the model was far too large to be efficient. The R-squared value wasn't bad, but it was somewhat of a misnomer. Adding many independent variables to a model in general creates a higher R-squared value, but the model is not optimized.

Checking the Variance Inflation Factor, it's clear that there exists strong colinearity within the model that must be accounted for. I decided to drop the zipcode, latitude, and longitude values because my project is not as concerned with geographic location as actual characteristics of the houses studied. 

Moving forward with model 2, I decided to focus on the bathrooms, sqft_living, and floors attributes. These attributes all had high correlations with price without extremely high VIF values. 

In [42]:
x2 = df1[["bedrooms", "sqft_living", "floors"]]
model2 = sm.OLS(endog=y, exog=sm.add_constant(x2)).fit()
model_summary2 = model2.summary()
model_summary2


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     7398.
Date:                Tue, 01 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:06:25   Log-Likelihood:            -2.9976e+05
No. Observations:               21597   AIC:                         5.995e+05
Df Residuals:                   21593   BIC:                         5.996e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        7.704e+04   7684.108     10.026      0.000     6.2e+04    9.21e+04
bedrooms    -5.775e+04   2324.412    -24.846      0.000   -6.23e+04   -5.32e+04
sqft_living   313.8304      2.467    127.187      0.000     308.994     318.667
floors       3476.4322   3480.340      0.999      0.318   -3345.290    1.03e+04
==============================================================================
Omnibus:                    14415.709   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           491526.502
Skew:                           2.733   Prob(JB):                         0.00
Kurtosis:                      25.723   Cond. No.                     1.04e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

We can see from the above data that the linear regression coefficient for "bedrooms" is actually negative. As we are looking at the price, we want to find positive correlations between features and the price of homes. In addition, if we use an alpha value of .05 for our models, the P-value for floors is greater than alpha. We can then determine floors is not a good predictor for the price of a home. 

# Adding Categorical Variables

I wanted to see if adding categorical variables would improve my model. However, categorical variables cannot be used in Linear Regression as is. I used OneHotEncoder to transform the dataset into a usable form for Linear Regression.

In [38]:
bc = data[["condition"]]
ohe1 = OneHotEncoder(sparse = False)
bcdf = pd.DataFrame(ohe1.fit_transform(bc))
bcdf.head()

combined_df = bcdf.join(df1[["price"]])
combined_df.columns

final = combined_df.drop([0],axis="columns")
final

x3 = final[[1,2,3,4]]
y = final[["price"]]
model3 = sm.OLS(endog=y, exog=sm.add_constant(x3)).fit()
model_summary3 = model3.summary()
model_summary3

#model4 = LinearRegression()
#x = final.drop("price", axis="columns")
#y = final["price"]


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     36.86
Date:                Tue, 01 Mar 2022   Prob (F-statistic):           9.12e-31
Time:                        12:45:28   Log-Likelihood:            -3.0732e+05
No. Observations:               21597   AIC:                         6.146e+05
Df Residuals:                   21592   BIC:                         6.147e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.422e+05   3092.358    175.327      0.000    5.36e+05    5.48e+05
1           -2.14e+05   2.83e+04     -7.574      0.000   -2.69e+05   -1.59e+05
2           -2.08e+04   5760.105     -3.611      0.000   -3.21e+04   -9508.375
3          -2.011e+05   6.81e+04     -2.955      0.003   -3.35e+05   -6.77e+04
4            7.04e+04   9401.080      7.489      0.000     5.2e+04    8.88e+04
==============================================================================
Omnibus:                    19184.771   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1160794.218
Skew:                           4.043   Prob(JB):                         0.00
Kurtosis:                      37.994   Cond. No.                         28.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

To avoid the Dummy Variable trap, I dropped one column from the dataset for the Condition attribute. We can see there is indeed a relationship between the condition of the home and the price of the home. 

For my fourth model, I will join the condiiton feature with the sqft_living feature to hopefully create a stronger model of the price in relation to the condition and the sqft_living. 

In [61]:
df4 = df1[["sqft_living"]].join(final)
#print(df4)

x4 = df4[["sqft_living",1,2,3,4]]
y = df4[["price"]]
model4 = sm.OLS(endog=y, exog=sm.add_constant(x4)).fit()
model_summary4 = model4.summary()
model_summary4

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     4307.
Date:                Tue, 01 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:49:26   Log-Likelihood:            -2.9992e+05
No. Observations:               21597   AIC:                         5.999e+05
Df Residuals:                   21591   BIC:                         5.999e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -6.577e+04   4713.495    -13.953      0.000    -7.5e+04   -5.65e+04
sqft_living   282.8679      1.941    145.756      0.000     279.064     286.672
1           -7958.4011   2.01e+04     -0.396      0.692   -4.74e+04    3.15e+04
2            3.515e+04   4107.505      8.558      0.000    2.71e+04    4.32e+04
3            5.384e+04   4.84e+04      1.113      0.266   -4.09e+04    1.49e+05
4            1.061e+05   6679.028     15.890      0.000     9.3e+04    1.19e+05
==============================================================================
Omnibus:                    14845.154   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           556085.012
Skew:                           2.825   Prob(JB):                         0.00
Kurtosis:                      27.208   Cond. No.                     6.22e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Great! As demonstrated above, I was able to implement both the continuous variable of sqft_living as well as the discrete variable of home condition to create a model of home price. However, the value of "3" in the condition subset fails the P-test. We must reject that condition has a strong affect on the price of the home. 

# Final Model

This brings us to our final model. The final model only includes sqft_living as a variable in relation to price. The R-squared value of .493% means that 49.3% of price fluctuation is covered by this one variable. The only other model that showed significantly higher R-squared values was our first model of all the numerical variables. However, this model was much too large to be practical. We can determine that sqft_living has the most impact on home values. 

In [62]:
x5 = df1[["sqft_living"]]
model2 = sm.OLS(endog=y, exog=sm.add_constant(x5)).fit()
model_summary5 = model5.summary()
model_summary5

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                 2.097e+04
Date:                Tue, 01 Mar 2022   Prob (F-statistic):               0.00
Time:                        14:50:01   Log-Likelihood:            -3.0006e+05
No. Observations:               21597   AIC:                         6.001e+05
Df Residuals:                   21595   BIC:                         6.001e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -4.399e+04   4410.023     -9.975      0.000   -5.26e+04   -3.53e+04
sqft_living   280.8630      1.939    144.819      0.000     277.062     284.664
==============================================================================
Omnibus:                    14801.942   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           542662.604
Skew:                           2.820   Prob(JB):                         0.00
Kurtosis:                      26.901   Cond. No.                     5.63e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Results

## Model 1
Model 1 accounted for all of the numerical variables in the dataset. Therefore the R-squared value was quite high (.627) as well as the dimension.Some standard error values were quite high as well as a number of negative coefficients. This model was not optimized.

## Model 2
Model 2 focused on the bedrooms, living space, and floors of the homes in the dataset. Although its R-squared was good (.507), the p-value of floors failed at an alpha level of .05. There was also a negative correlation between bedrooms and price. 

## Model 3
Model 3 took into account the condition value of the homes in the dataset. The conditions ranged from 0-Poor to 5-Very good. There was a small R-value of 0.007, which means that the condition of these homes accounted for less than 1% of price variation. Even though this value was small, I worked it into my fourth model to see if it could improve Model 2.

## Model 4
Model 4 looked at the relationship between Price, Condition, and Living Space. The R-squared was slightly improved from Living Space v. Price alone, but several variables failed by having a p-value of greater than and alpha of .05. Therefore, this model was not acceptable. 

## Final Model
Dissapointing as it may seem, I found that the best model was just Living Space vs. Price. This model takes into account 49.3% of the variation in Price just from living space alone. 


Linear Regression dives deeper into the data than just a graph. If you simply plotted bedrooms or floors versus price, it may seem as though there is an impact. However, with Linear Regression, we can see this is not the case. 

Some limitations in this analysis include uncertainty in the grading of the condition of the home. It also looks at homes in a wide variety of zipcodes and locales, from Seattle to rural areas. 

# Recommendations
Lynn Crane Real Estate should prioritize houses with large living space over all other factors in purchasing decisions. Although it seems intuitive that more bedrooms would raise the price of the house, our data does not suggest that this is a significant factor. In addition, the number of floors does not have a significant effect on the price of the house. The condition of the house, from 0-Poor to 5-Very Good, does not have a significantly impact on price either. 

- Find homes with lots of living space that are undervalued, even if their condition is not top quality
- Prioritize living space over number of floors in a prospective hosue
- If a house can be rennovated for resale, advise the homeowner to expand the living space. 
